In [23]:
import sqlite3
import re

In [24]:
import hashlib

In [25]:
import pyotp

In [26]:
conn = sqlite3.connect('Users.db')

cursor = conn.cursor()

In [27]:
cursor.execute(''' CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY AUTOINCREMENT, username TEXT, password TEXT, 
password_hash TEXT, mfa_secret TEXT)''')

In [29]:
# Prompt user for username and password
username = input("Enter your username: ")

while True:
    # Prompt user for password
    password = input("Enter your password: ")

    # Check if password meets complexity requirements
    if (len(password) < 8 or not re.search("[a-z]", password) or not re.search("[A-Z]", password) or not re.search("[0-9]", password) or not re.search("[!@#$%^&*()_+]", password)):
        print("Password should be at least 8 characters long and contain at least one uppercase letter, one lowercase letter, one digit, and one special character (!@#$%^&*()_+). Please try again.")
    else:
        break

# Generate a hash of the password using SHA-256
password_hash = hashlib.sha256(password.encode()).hexdigest()


# Generate a random MFA secret key using the PyOTP library
mfa_secret = pyotp.random_base32()

# Insert the username and password into the users table
cursor.execute('INSERT INTO users (username, password,  mfa_secret) VALUES (?, ?, ?)', (username, password_hash, mfa_secret))



In [ ]:
# *=> all 
query = "SELECT * FROM users"
cursor.execute(query).fetchall()

In [31]:
# Commit the transaction and close the connection
conn.commit()
conn.close()

In [37]:
# Prompt user to set up MFA using the secret key
print("Your MFA secret key is:", mfa_secret)
print("Please scan the QR code below with your MFA app:")
print(pyotp.totp.TOTP(mfa_secret).provisioning_uri(name=username, issuer_name='MyApp'))

# Prompt user to enter their MFA code
mfa_code = input("Enter your MFA code: ")

# Verify the MFA code using PyOTP
totp = pyotp.TOTP(mfa_secret)
if not totp.verify(mfa_code):
    print("Invalid MFA code. Please try again.")


Your MFA secret key is: 2J2HMGQZU42CFJQAQ5FKG2FD4I7WRU73
Please scan the QR code below with your MFA app:
otpauth://totp/MyApp:Ali?secret=2J2HMGQZU42CFJQAQ5FKG2FD4I7WRU73&issuer=MyApp
